In [ ]:
!pip install pandas
!pip install dotenv
%pip install PyYAML


In [ ]:
# load csv
import pandas as pd

# Load the CSV file
df = pd.read_csv('data.csv')

# drop null values in shipping_postal_code and shipping_province
df = df.dropna()
print(f"DataFrame shape after dropping null values: {df.shape}")

In [ ]:
# import the YAML file with zip prefixes
from dotenv import load_dotenv
import os
from pprint import pprint

load_dotenv()

# load the YAML file
import yaml

with open(os.getenv('PATH_TO_YAML') or "zip_prefixes.yaml", 'r') as file:
    zip_prefixes = yaml.safe_load(file)

pprint(zip_prefixes)

In [ ]:
# map zone names to zone codes
name_to_code = {}
for zone in zip_prefixes["zones"]:
    name_to_code[zone["name"]] = zone["code"]
    for n in zone["name_alternates"]:
        name_to_code[n] = zone["code"]

pprint(name_to_code)

In [ ]:
# map zone codes to allowed prefixes
allowed_prefixes_for_zone_code = {}
for zone in zip_prefixes["zones"]:
    allowed_prefixes_for_zone_code[zone["code"]] = set(zone["zip_prefixes"])

In [ ]:
import re
from collections import defaultdict
# get data rows from pandas df
rows = []
unmapped_cities = defaultdict(int)

for _, row in df.iterrows():
    if row["shipping_province"] not in name_to_code:
        unmapped_cities[row["shipping_province"]] += 1
        continue
    zone_code, postal_code = name_to_code[row["shipping_province"]], row["shipping_postal_code"]
    if not isinstance(postal_code, str):
        continue
    rows.append({
        "zone_code": zone_code,
        "postal_code": postal_code,
    })

regexp = "^[0-9]{3}-[0-9]{4}$"
rows = list(filter(lambda x: re.match(regexp, x["postal_code"]), rows)) # filter out rows that don't match the format
print("Total rows: ", len(rows))
rows

In [ ]:
#validate the zip prefixes
invalid_prefixes = defaultdict(list)
for row in rows:
    zone_code, postal_code = row["zone_code"], row["postal_code"]
    zone_prefixes = allowed_prefixes_for_zone_code[zone_code]
    match = False
    for i in range(len(postal_code)):
        if postal_code[:i] in zone_prefixes:
            match = True
            continue
    if not match:
        invalid_prefixes[zone_code].append(postal_code)

print("Total rows: ", len(rows))
print("Total invalid prefixes: ", sum(len(v) for v in invalid_prefixes.values()))
print("Total unique invalid prefixes: ", len(set(sum(invalid_prefixes.values(), []))))
print("Percentage of invalid prefixes: ", sum(len(v) for v in invalid_prefixes.values()) / len(rows))

invalid_prefixes
